## TAREA

* Con el código utilizado en clase, implementar una clase de Python similar a esta. Se le deben pasar como argumentos:
  - Cantidad de filtros
  - Tamaño de los filtros
  - Stride
  - Tipo de padding (“valid” o “same”)
* Debe tener un método “forward”, al cual se le pasa la matriz de entrada y devuelve la salida.
*Y un método “backward”, al cual se le pasa una matriz con $∂L/∂O$ y devuelve dos matrices, una con $∂L/∂F$ y otra con $∂L/∂X$.

In [1]:
import numpy as np
import torch

In [ ]:
# Función para padding con ceros

def zero_padding(X, pad):
  """
  Argumentos:
    X: Array numpy de entrada con dimensiones (batch_size, n_C, n_H, n_W)
    pad: Entero representando la cantidad de filas y columnas que se deben agregar con ceros

  Retorna:
    X_padded: Array numpy con dimensiones (batch_size, n_C, n_H + 2*pad, n_W + 2*pad)
  """

  X_padded = np.pad(X, ((0,0), (0,0), (pad, pad), (pad, pad)), mode='constant', constant_values = (0,0))

  return X_padded

In [2]:
# Función para realizar la operación de convolución

def convolve(X, W, b):
  """
  Argumentos:
    X: Array numpy de entrada con dimensiones (filter_size, filter_size, n_C_prev)
    W: Array numpy con los pesos de un filtro con dimensiones (filter_size, filter_size, n_C_prev)
    b: Entero con el valor de bias de la capa actual

  Retorna:
    Z: Entero con el valor del resultado
  """

  # Multiplico elemento a elemento el valor de entrada con los pesos del filtro
  aux = X * W
  # Realizo la suma de todos los elementos
  aux = np.sum(aux)
  # Le sumo el valor del bias para obtener Z
  Z = aux + float(b)

  return Z

In [3]:
# Función para realizar el forward pass de una capa convolucional

def conv_forward(layer_input, W, b, stride, padding):
  """
  Argumentos:
    layer_input: Array numpy con los valores de entrada a la capa convolucional (batch_size, n_C_prev, n_H_prev, n_W_prev)
    W: Array numpy con los pesos de los filtros utilizados en la capa actual (n_C, n_C_prev, filter_size, filter_size)
    b: Array numpy con los valores de bias utilizados en la capa actual (1, 1, 1, n_C)
    stride: Entero con el valor de stride utilizado en la capa actual.
    padding: 

  Retorna:
    Z: Array numpy con los valores de salida de la capa convolucional (batch_size, n_H, n_W, n_C)
  """

  # Obtengo las dimensiones de la entrada
  (batch_size, n_C_prev, n_H_prev, n_W_prev) = layer_input.shape

  # Obtengo las dimensiones de los filtros
  (n_C, n_C_prev, filter_size, filter_size) = W.shape

  # Calculo las dimensiones del volumen de salida de la capa actual
  n_H = int((n_H_prev + 2*padding - filter_size)/stride + 1)
  n_W = int((n_W_prev + 2*padding - filter_size)/stride + 1)

  # Inicializo el volumen de salida con ceros
  Z = np.zeros([batch_size, n_C, n_H, n_W])

  # Agrego padding con ceros al volumen de entrada
  layer_input_padded = zero_padding(layer_input, padding)

  # Comienzo iterando sobre cada ejemplo del batch
  for i in range(batch_size):

    # Itero sobre el eje vertical del volumen de salida
    for h in range(n_H):
      # Calculo las coordenadas verticales de inicio y fin de la ventana sobre la que aplicaremos el filtro
      y_start = stride * h
      y_end = y_start + filter_size

      # Itero sobre el eje horizontal del volumen de salida
      for w in range(n_W):
        # Calculo las coordenadas horizontales de inicio y fin de la ventana sobre la que aplicaremos el filtro
        x_start = stride * w
        x_end = x_start + filter_size

        # Extraigo la ventana para calcular la convolucion, del volumen de entrada con padding
        slice_from_input_padded = layer_input_padded[i, :, y_start:y_end, x_start:x_end]
        
        # Itero sobre la cantidad de canales del volumen de salida
        for c in range(n_C):

          # Obtengo el valor del filtro y bias del canal correspondiente
          filter = W[c, :, :, :]
          bias = b[c]

          # Computo la operación de convolución para esta ventana
          Z[i, c, h, w] = convolve(slice_from_input_padded, filter, bias)
  
  return Z

In [5]:
def max_pooling_forward(layer_input, filter_size, stride):
  """
  Argumentos:
    layer_input: Array numpy con los valores de entrada a la capa max-pooling (batch_size, n_C_prev, n_H_prev, n_W_prev)
    filter_size: Entero con el valor de tamaño de filtro utilizado en la capa actual. 
    stride: Entero con el valor de stride utilizado en la capa actual.

  Retorna:
    Z: Array numpy con los valores de salida de la capa max-pooling (batch_size, n_C, n_H, n_W)
  """

  # Obtengo las dimensiones de la entrada
  (batch_size, n_C_prev, n_H_prev, n_W_prev) = layer_input.shape

  # Calculo las dimensiones del volumen de salida de la capa actual
  n_H = int(1 + (n_H_prev - filter_size) / stride)
  n_W = int(1 + (n_W_prev - filter_size) / stride)
  n_C = n_C_prev

  # Inicializo el volumen de salida con ceros
  Z = np.zeros([batch_size, n_C, n_H, n_W])

  # Comienzo iterando sobre cada ejemplo del batch
  for i in range(batch_size):

    # Itero sobre el eje vertical del volumen de salida
    for h in range(n_H):
      # Calculo las coordenadas verticales de inicio y fin de la ventana
      y_start = stride * h
      y_end = y_start + filter_size

      # Itero sobre el eje horizontal del volumen de salida
      for w in range(n_W):
        # Calculo las coordenadas horizontales de inicio y fin de la ventana
        x_start = stride * w
        x_end = x_start + filter_size
        
        # Itero sobre la cantidad de canales del volumen de salida
        for c in range(n_C):

          # Obtengo el maximo
          Z[i, c, h, w] = np.max(layer_input[i, c, y_start:y_end, x_start:x_end])

  return Z

In [6]:
def average_pooling_forward(layer_input, filter_size, stride):
  """
  Argumentos:
    layer_input: Array numpy con los valores de entrada a la capa max-pooling (batch_size, n_C_prev, n_H_prev, n_W_prev)
    filter_size: Entero con el valor de tamaño de filtro utilizado en la capa actual. 
    stride: Entero con el valor de stride utilizado en la capa actual.

  Retorna:
    Z: Array numpy con los valores de salida de la capa average-pooling (batch_size, n_C, n_H, n_W)
  """

  # Obtengo las dimensiones de la entrada
  (batch_size, n_C_prev, n_H_prev, n_W_prev) = layer_input.shape

  # Calculo las dimensiones del volumen de salida de la capa actual
  n_H = int(1 + (n_H_prev - filter_size) / stride)
  n_W = int(1 + (n_W_prev - filter_size) / stride)
  n_C = n_C_prev

  # Inicializo el volumen de salida con ceros
  Z = np.zeros([batch_size, n_C, n_H, n_W])

  # Comienzo iterando sobre cada ejemplo del batch
  for i in range(batch_size):

    # Itero sobre el eje vertical del volumen de salida
    for h in range(n_H):
      # Calculo las coordenadas verticales de inicio y fin de la ventana
      y_start = stride * h
      y_end = y_start + filter_size

      # Itero sobre el eje horizontal del volumen de salida
      for w in range(n_W):
        # Calculo las coordenadas horizontales de inicio y fin de la ventana
        x_start = stride * w
        x_end = x_start + filter_size
        
        # Itero sobre la cantidad de canales del volumen de salida
        for c in range(n_C):

          # Obtengo el maximo
          Z[i, c, h, w] = np.mean(layer_input[i, c, y_start:y_end, x_start:x_end])

  return Z